In [2]:
# Import required libraries
import numpy as np
import pandas as pd
import scipy
from scipy import stats
import statsmodels.stats.multitest as smm
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import seaborn as sns

pd.set_option('display.precision', 2)
pd.set_option('display.max_columns',10)


### Start by importing our expression dataset

We're going to start by importing data the expression-metadata merged spreadsheet, so that we've got something to work with.

In [3]:
import pandas as pd

# Upload the file containing melanoma dataset 
df = pd.read_excel(r"C:\Users\Kathleen\OneDrive\Documents\Bioinformatics Spring 2024\Code\Class 5\melanoma_zerosRemoved_log2transformed_2024.xlsx")




In [4]:
# importing the melanoma dataset
melanoma_log2 = pd.read_excel('melanoma_zerosRemoved_log2transformed_2024.xlsx',index_col = 0)
melanoma_log2.head()

,Sample_geo_accession,Stage,cell type,sample_number,cell_line,...,ZYX,ZZEF1,ZZZ3,BP-21201H5.1,BP-2189O9.2
Sample Title,,,,,,,,,,,
FM_1,GSM2344965,primary melanocytes,normal melanocytes,1,FM,...,11.66,11.83,10.55,3.17,1.00
FM_2,GSM2344966,primary melanocytes,normal melanocytes,2,FM,...,11.63,11.54,11.06,2.81,2.00
FM_3,GSM2344967,primary melanocytes,normal melanocytes,3,FM,...,12.31,11.44,10.81,2.32,1.58
SK_MEL_28_1,GSM2344968,metastatic,melanoma cell line,4,SK_MEL_28,...,11.86,10.94,11.64,0.00,6.04
SK_MEL_28_2,GSM2344969,metastatic,melanoma cell line,5,SK_MEL_28,...,11.97,11.03,11.74,1.00,6.41


In [6]:
# Importing two gene lists
AdipogenesisGenes = pd.read_csv('Hallmark_Adipogenesis_geneset.txt', sep="/t",engine='python')
AdipogenesisGenes.head()

,HALLMARK_ADIPOGENESIS
0,ABCA1
1,ABCB8
2,ACAA2
3,ACADL
4,ACADM


1. Calculate the top 1000 genes with the highest (absolute value) difference in mean log2 expression between metastatic vs. normal, and output to an excel spreadsheet. **Hint**: it might be helpful to prepare a data frame that only contains the expression values and the Stage label information. There are several ways to do this.

In [7]:
# Step 1: Create a new DataFrame with expression values and Stage label
expression_df = df.drop(columns=['Sample Title', 'Sample_geo_accession', 'cell type', 'sample_number', 'cell_line'])
expression_df.head()

# Step 2: Calculate the mean log2 expression for metastatic and normal samples
metastatic_mean = expression_df.loc[df['Stage'] == 'metastatic'].drop(columns=['Stage']).mean()
normal_mean = expression_df.loc[df['Stage'] == 'primary melanocytes'].drop(columns=['Stage']).mean()

# Step 3: Calculate the absolute difference in mean log2 expression
diff_mean = abs(metastatic_mean - normal_mean)

# Step 4: Sort the genes based on the absolute difference
sorted_genes = diff_mean.sort_values(ascending=False)

# Step 5: Select the top 1000 genes with the highest absolute difference
top_1000_genesHW3 = sorted_genes.head(1000)
top_1000_genesHW3.head()

# Step 6: Output the selected genes to an Excel spreadsheet
top_1000_genesHW3.to_excel('top_1000_genesHW3KAPABIO536.xlsx')


2. Use http://bioinformatics.sdstate.edu/go/ to perform enrichment analysis on these top 1000 genes and report back what the top enriched KEGG pathway is. Please be sure to take a look at the outputs of the other tabs from ShinyGO, but no need to report back anything other than the name of the top enriched KEGG pathway.

In [9]:
import pandas as pd

# Upload the enrichment.csv file
enrichment_df = pd.read_csv(r"C:\Users\Kathleen\OneDrive\Documents\Bioinformatics Spring 2024\Homework\Homework 3\enrichment.csv")
enrichment_df.head()



,Enrichment FDR,nGenes,Pathway Genes,Fold Enrichment,Pathway,URL,Genes
0,1.48e-04,15,88,4.49,Path:hsa04512 ECM-receptor interaction,http://www.genome.jp/kegg-bin/show_pathway?hsa...,CHAD COL1A2 COL6A1 COL9A1 TNC FREM2 ITG...
1,3.88e-02,6,36,4.39,Path:hsa00512 Mucin type O-glycan biosynthesis,http://www.genome.jp/kegg-bin/show_pathway?hsa...,ST6GALNAC2 GALNT15 GALNT3 GCNT1 GALNTL6 ...
2,3.00e-02,7,45,4.10,Path:hsa02010 ABC transporters,http://www.genome.jp/kegg-bin/show_pathway?hsa...,ABCC9 ABCA8 ABCA13 ABCA1 ABCB5 ABCB4 ABCG4
3,6.41e-03,12,89,3.55,Path:hsa04727 GABAergic synapse,http://www.genome.jp/kegg-bin/show_pathway?hsa...,ADCY2 ADCY5 ABAT GABRP GAD1 GLUL GNAO1 ...
4,1.86e-02,10,77,3.42,Path:hsa05412 Arrhythmogenic right ventricular...,http://www.genome.jp/kegg-bin/show_pathway?hsa...,DSC2 DSG2 ITGA6 ITGA4 LAMA2 PKP2 RYR2 ...


The top enriched KECG Pathway is the hsa04512 ECM-receptor interaction. 

3. Extract the genes that overlap between the `AdipogenesisGenes` and the top 1000 genes with the greatest difference in mean log2 expression change in `melanoma_log2` that you found in question 1. **Hint**: [This link](https://www.geeksforgeeks.org/python-intersection-two-lists/) should be helpful.

In [12]:
import numpy as np

# Extract the gene names from AdipogenesisGenes and top_1000_genesHW3
adipogenesis_genes = AdipogenesisGenes['HALLMARK_ADIPOGENESIS'].tolist()
top_1000_genes = top_1000_genesHW3.index.tolist()

# Find the overlapping genes
overlapping_genes = np.intersect1d(adipogenesis_genes, top_1000_genes)

# Print the overlapping genes
print("The genes which overlap between 'AdipogenesisGenes' and 'melanoma_log2' are:",overlapping_genes)


The genes which overlap between 'AdipogenesisGenes' and 'melanoma_log2' are: ['ABCA1' 'ALDH2' 'APOE' 'ENPP2' 'GPX3' 'ITIH5' 'LIFR' 'LIPE' 'MGLL'
 'SORBS1' 'SSPN' 'STAT5A']


4. Compare Pearson correlation coefficients between each pair of genes that you found in question 3 for metastatic vs. normal samples. Which gene pair has the most negative difference in Pearson coefficient when you subtract the coefficients calculated from the normal values from the coefficients calculated from the metastatic values? **Hint** use the `.unstack()` and `.sort_values()` methods to help you sort the data frame matrix.

In [13]:
# Reiteration of expression_df from Question 1

expression_df = df.drop(columns=['Sample Title', 'Sample_geo_accession', 'cell type', 'sample_number', 'cell_line'])
expression_df.head()

# The Overlapping genes are: ['ABCA1' 'ALDH2' 'APOE' 'ENPP2' 'GPX3' 'ITIH5' 'LIFR' 'LIPE' 'MGLL'
 #'SORBS1' 'SSPN' 'STAT5A']

 # Metastatic samples are rows for which the 'Stage' column value is 'metastatic'; normal samples are rows for which the 'Stage' column value is 'primary melanocytes'

,Stage,A1BG,A2M,AAAS,AACS,...,ZYX,ZZEF1,ZZZ3,BP-21201H5.1,BP-2189O9.2
0,primary melanocytes,8.64,14.51,10.54,10.79,...,11.66,11.83,10.55,3.17,1.00
1,primary melanocytes,8.32,15.54,10.42,11.01,...,11.63,11.54,11.06,2.81,2.00
2,primary melanocytes,8.94,14.32,10.63,11.42,...,12.31,11.44,10.81,2.32,1.58
3,metastatic,8.37,11.37,10.76,10.70,...,11.86,10.94,11.64,0.00,6.04
4,metastatic,8.50,11.40,10.91,10.74,...,11.97,11.03,11.74,1.00,6.41


In [14]:
import numpy as np
import pandas as pd

# Select the overlapping genes
overlapping_genes = ['ABCA1', 'ALDH2', 'APOE', 'ENPP2', 'GPX3', 'ITIH5', 'LIFR', 'LIPE', 'MGLL', 'SORBS1', 'SSPN', 'STAT5A']

# Filter the expression dataframe for the overlapping genes and metastatic/normal samples
metastatic_samples = expression_df[expression_df['Stage'] == 'metastatic'][overlapping_genes]
normal_samples = expression_df[expression_df['Stage'] == 'primary melanocytes'][overlapping_genes]

# Calculate the correlation coefficients for each pair of genes separately for metastatic and normal samples
metastatic_corr = metastatic_samples.corr()
normal_corr = normal_samples.corr()

# Subtract the coefficients calculated from the normal values from the coefficients calculated from the metastatic values
corr_diff = metastatic_corr - normal_corr

# Find the gene pair with the most negative difference in Pearson coefficient
max_diff_gene_pair = corr_diff.unstack().sort_values().index[0]

# Get the difference value for the gene pair
diff_value = corr_diff.loc[max_diff_gene_pair]

# Print the gene pair with the most negative difference in Pearson coefficient
print("Gene pair with the most negative difference in Pearson coefficient:", max_diff_gene_pair)
print("Difference value:", diff_value)


Gene pair with the most negative difference in Pearson coefficient: ('ABCA1', 'APOE')
Difference value: -1.9104565893356764


5. Please write a few sentences about the dataset that you currently plan to use for your final project. If it's a dataset from your labwork, then please describe what the samples and the measurements are, as well as what metadata are available. If it's a paper, please provide the citation, and describe what the samples and the measurements are. Note: what you write here is not binding (you can definitely change to a different dataset later if you wish), but we want to get a sense for your current thoughts on the project.

I've devoted time to choosing the dataset for my Final Project. Following thorough evaluation, I've decided on the dataset featured in the publication "Transcriptomic profile of circulating memory CD4 T cells can differentiate between latent tuberculosis individuals and healthy controls" by Burel, et al. Given my background as a microbiologist with experience in tuberculosis diagnostics in both outpatient and inpatient settings, and my emerging skills in bioinformatics, I believe this project will be an exciting way to apply everything I've learned in this class! Citation: 	
Burel JG, Lindestam Arlehamn CS, Khan N, Seumois G et al. Transcriptomic Analysis of CD4(+) T Cells Reveals Novel Immune Signatures of Latent Tuberculosis. J Immunol 2018 May 1;200(9):3283-3290. PMID: 29602771Link: https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE99373